## Load and prep data

In [101]:
import pandas as pd
import numpy as np

import nltk
import re
import heapq

In [99]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\suzie\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [111]:
data = pd.read_csv('final_cohort_with_study_paths.csv', index_col=0)

In [153]:
data.head()

,subject_id,hadm_id,48h_hf,14d_hf,30d_hf,er_hf,48h,14d,30d,er,study_id,last_dicom_id,mimic_id,path,study_text,word_counts
0,10003400,20214994,0,0,0,0,0,0,0,0,52437868,2f6a5fc9-40af95f8-d8762332-f56005ea-b5f85cc2,p10003400_s52437868_2f6a5fc9-40af95f8-d8762332...,files/p10/p10003400/s52437868.txt,final report examination chest portable ap ind...,"[[10, 5, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1..."
1,10011938,22624746,0,0,0,1,0,0,0,1,56362279,c6eed867-d6efb38a-438501f9-9d2506e9-a0c958f8,p10011938_s56362279_c6eed867-d6efb38a-438501f9...,files/p10/p10011938/s56362279.txt,final report indication ___f with h o complex ...,"[[9, 6, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2,..."
2,10011938,23501236,0,0,0,0,0,0,0,1,51895247,bf724128-9131f33a-6fd065d5-19041750-9e7f8707,p10011938_s51895247_bf724128-9131f33a-6fd065d5...,files/p10/p10011938/s51895247.txt,final report examination chest pa and lat indi...,"[[5, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1,..."
3,10012853,26369609,0,0,0,0,0,0,0,1,58181999,aac02704-c647b84c-58d5df12-c9857852-e1536bba,p10012853_s58181999_aac02704-c647b84c-58d5df12...,files/p10/p10012853/s58181999.txt,final report examination chest portable ap ind...,"[[5, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,..."
4,10015931,28157142,0,0,1,1,0,0,1,1,57792054,094f66e7-fc7faa70-d5f0b6b5-5a8b82f6-58d284b8,p10015931_s57792054_094f66e7-fc7faa70-d5f0b6b5...,files/p10/p10015931/s57792054.txt,wet read ___ ___ ___ 10 53 pm moderate bilater...,"[[6, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,..."


In [112]:
report_base_dir = "C:/Users/suzie/Dropbox (MIT)/Spring 2021/6.871 Machine Learning for Healthcare/mimic-cxr-reports/"

data['study_text'] = data['path'].apply(lambda path: pd.read_csv(report_base_dir+path, sep="\t", header=None).to_numpy().flatten())
data['study_text'] = data['study_text'].apply(lambda text: ' '.join(text)).str.strip()

In [123]:
data['study_text'][1]

'final report indication ___f with h o complex seizure disorder related to l frontal avm h o aca aneurysm clipped copd fev ___ transferred to ___ from ___ on ___ where she was admitted on ___ p w dizziness and fall s p vats biopsy of lung hypereosinophilic syndrome new o2 requirement eval for pulm edema or etiology of hypoxemia technique chest pa and lateral comparison chest ct dated ___ findings lines and tubes none lungs the lungs are well inflated and demonstrate increased interstitial markings and haziness in bilateral mid and lower zones surgical sutures project over the left apex pleura there is no pleural effusion or pneumothorax mediastinum there is cardiomegaly and unfolding of the thoracic aorta bony thorax unremarkable impression increased interstitial markings and haziness in bilateral mid and lower zones likely a combination of interstitial process with underlying pulmonary edema there is no lobar consolidation '

In [122]:
# convert text to lower case, remove non-word characters, remove punctuation
data['study_text'] = data['study_text'].str.lower()
data['study_text'] = data['study_text'].apply(lambda text: re.sub(r'\W', ' ', text))
data['study_text'] = data['study_text'].apply(lambda text: re.sub(r'\s', ' ', text))

In [ ]:
### should also remove numbers, __'s from study_text

In [144]:
## get dict carrying all words across all text

reports = data['study_text'].to_numpy()

word2count = {}
for report in reports:
    words = nltk.word_tokenize(report)
    for word in words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1

In [ ]:
# data['word_counts'] = data['study_text'].apply(lambda text: [pd.value_counts(nltk.word_tokenize(text))])

In [149]:
#take the top n words, here n = 100
freq_words = heapq.nlargest(100, word2count, key=word2count.get)

final_vec = []
for report in reports:
    vector = []
    for word in freq_words:
        if word in nltk.word_tokenize(report):
#             vector.append(#number of times word appears in report)
            vector.append(1)
        else:
            vector.append(0)
    final_vec.append(vector)
final_vec = np.asarray(final_vec)
#returns a 2D array #reports x vector


In [150]:
final_vec

[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 1 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 1 ... 0 0 1]]


In [151]:
final_vec.shape

(2594, 100)

## Logistic Regression

In [201]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [231]:
#train test split

X = final_vec
y = list(data['14d_hf'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [232]:
logreg_48h_hf = LogisticRegression(random_state = 0, class_weight='balanced')
logreg_48h_hf.fit(X_train, y_train)

C:\Users\suzie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=0)

In [233]:
y_pred = logreg_48h_hf.predict(X_test)

In [234]:
acc = np.mean(y_pred == y_test)*100
print ('Accuracy = %.0f%%' %acc)

Accuracy = 64%


In [235]:
# accuracies 48h = 99%, 14d = 93%, 30d_hf = 87%, er_hf = 80%, 14h = 98%, 14d = 85%, 30d = 73%, er = 55%

In [236]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.66      0.77       482
           1       0.06      0.27      0.10        37

    accuracy                           0.64       519
   macro avg       0.49      0.47      0.43       519
weighted avg       0.86      0.64      0.72       519



In [237]:
confusion_matrix(y_test, y_pred)

array([[320, 162],
       [ 27,  10]], dtype=int64)